## Analisis exploratorio de datos de NLP

In [1]:
import pandas as pd
import numpy as np

In [2]:
movie_df = pd.read_csv('movies_final_all.csv')
#dejamos caer las columnas que contienen los datos que no vamos a usar
movie_df.drop('Unnamed: 0', axis=1, inplace=True)
movie_df.drop('traduccion_idioma', axis=1, inplace=True)
movie_df

,titulo,link,traduccion
0,Apocalypse Now,https://www.imdb.com/title/tt0078788/?ref_=adv...,'Un oficial del ejército estadounidense que si...
1,El lago azul,https://www.imdb.com/title/tt0080453/?ref_=adv...,"'En la época victoriana, dos niños naufragan e..."
2,"Alien, el octavo pasajero",https://www.imdb.com/title/tt0078748/?ref_=adv...,'La tripulación de una nave espacial comercial...
3,El resplandor,https://www.imdb.com/title/tt0081505/?ref_=adv...,'Una familia se dirige a un hotel aislado para...
4,El Imperio contraataca,https://www.imdb.com/title/tt0080684/?ref_=adv...,'Tras ser brutalmente dominados los rebeldes p...
...,...,...,...
247141,Scream of the Bikini,https://www.imdb.com/title/tt1358254/?ref_=adv...,'Una joven pareja de Nueva York crea una intri...
247142,Puck Hogs,https://www.imdb.com/title/tt1249307/?ref_=adv...,'Un joven novelista intenta escribir sobre el ...
247143,Caído de las nubes,https://www.imdb.com/title/tt1526741/?ref_=adv...,'Padre e hijo viajando de Victoria a Cape York...
247144,Los Tres Investigadores y el Secreto del Casti...,https://www.imdb.com/title/tt1156519/?ref_=adv...,'Al Lcpl Arne Christenson le quedan tres días ...


Analisis exploratorio de datos antes de limpieza (número de carácteres, número de palabras)

In [3]:
#dejamos caer las filas que contienen valores nulos
movie_df.dropna(subset = ["traduccion"], inplace=True)
#el número de carácteres
movie_df['trad_num_car'] = movie_df['traduccion'].str.len()
#el número de palabras
movie_df['trad_num_pal'] = movie_df['traduccion'].str.split().str.len()
#count.index = count.index.astype(str)
movie_df.sort_index(inplace=True)
movie_df

,titulo,link,traduccion,trad_num_car,trad_num_pal
0,Apocalypse Now,https://www.imdb.com/title/tt0078788/?ref_=adv...,'Un oficial del ejército estadounidense que si...,174,31
1,El lago azul,https://www.imdb.com/title/tt0080453/?ref_=adv...,"'En la época victoriana, dos niños naufragan e...",206,34
2,"Alien, el octavo pasajero",https://www.imdb.com/title/tt0078748/?ref_=adv...,'La tripulación de una nave espacial comercial...,134,20
3,El resplandor,https://www.imdb.com/title/tt0081505/?ref_=adv...,'Una familia se dirige a un hotel aislado para...,206,32
4,El Imperio contraataca,https://www.imdb.com/title/tt0080684/?ref_=adv...,'Tras ser brutalmente dominados los rebeldes p...,228,35
...,...,...,...,...,...
247141,Scream of the Bikini,https://www.imdb.com/title/tt1358254/?ref_=adv...,'Una joven pareja de Nueva York crea una intri...,86,14
247142,Puck Hogs,https://www.imdb.com/title/tt1249307/?ref_=adv...,'Un joven novelista intenta escribir sobre el ...,160,28
247143,Caído de las nubes,https://www.imdb.com/title/tt1526741/?ref_=adv...,'Padre e hijo viajando de Victoria a Cape York...,130,24
247144,Los Tres Investigadores y el Secreto del Casti...,https://www.imdb.com/title/tt1156519/?ref_=adv...,'Al Lcpl Arne Christenson le quedan tres días ...,148,29


In [4]:
print("Número promedio de carácteres: " + str(movie_df['trad_num_car'].mean(axis = 0, skipna = True)))
print("Número promedio de palabras:   " + str(movie_df['trad_num_pal'].mean(axis = 0, skipna = True)))

Número promedio de carácteres: 227.3012753595041
Número promedio de palabras:   38.7719607033899


## Limpieza de texto
Para poder extraer las características vectoriales del corpus es conveniente realizar primero una limpieza y pre-procedado de cada descripción traducida.\
Realizaremos el siguiente procesado:
- Introducir un espacio después de determinados signos de puntuación (".", "?") para que el tokenizado sea correcto
- Separar el texto en *tokens*
- Eliminar signos de puntuación o espacios o de longitud 1
- Lematizar el texto y pasarlo a minúsculas

In [5]:
import spacy
import os, re

nlp = spacy.load("es_core_news_sm")

def normaliza(texto):
    #separamos después de ciertos signos de puntuación
    texto = re.sub(r"([\.\?])", r"\1 ", texto)
    doc = nlp(texto)
    # quitamos puntuación/espacios
    tokens = [t for t in doc if not t.is_punct and not t.is_space and len(t.text)>1]
    palabras = []
    for t in tokens:
        palabras.append(t.lemma_.lower())
    # juntamos de nuevo en una cadena
    salida = ' '.join(palabras)
    
    return salida

In [6]:
%%time

#normalizamos la descripción
movie_df['traduccion_limpia'] = movie_df['traduccion'].apply(normaliza)
movie_df

Wall time: 41min 26s


,titulo,link,traduccion,trad_num_car,trad_num_pal,traduccion_limpia
0,Apocalypse Now,https://www.imdb.com/title/tt0078788/?ref_=adv...,'Un oficial del ejército estadounidense que si...,174,31,uno oficial del ejército estadounidense que se...
1,El lago azul,https://www.imdb.com/title/tt0080453/?ref_=adv...,"'En la época victoriana, dos niños naufragan e...",206,34,en el época victorián dos niño naufragar en un...
2,"Alien, el octavo pasajero",https://www.imdb.com/title/tt0078748/?ref_=adv...,'La tripulación de una nave espacial comercial...,134,20,el tripulación de uno nave espacial comercial ...
3,El resplandor,https://www.imdb.com/title/tt0081505/?ref_=adv...,'Una familia se dirige a un hotel aislado para...,206,32,uno familia él dirigir uno hotel aislado para ...
4,El Imperio contraataca,https://www.imdb.com/title/tt0080684/?ref_=adv...,'Tras ser brutalmente dominados los rebeldes p...,228,35,tras ser brutalmente dominado el rebelde por e...
...,...,...,...,...,...,...
247141,Scream of the Bikini,https://www.imdb.com/title/tt1358254/?ref_=adv...,'Una joven pareja de Nueva York crea una intri...,86,14,uno joven pareja de nueva york creer uno intri...
247142,Puck Hogs,https://www.imdb.com/title/tt1249307/?ref_=adv...,'Un joven novelista intenta escribir sobre el ...,160,28,uno joven novelista intentar escribir sobre el...
247143,Caído de las nubes,https://www.imdb.com/title/tt1526741/?ref_=adv...,'Padre e hijo viajando de Victoria a Cape York...,130,24,padre hijo viajar de victoria cape york para c...
247144,Los Tres Investigadores y el Secreto del Casti...,https://www.imdb.com/title/tt1156519/?ref_=adv...,'Al Lcpl Arne Christenson le quedan tres días ...,148,29,al lcpl arne christenson él quedar tres día pa...


In [7]:
# guardamos el archivo
movie_df.to_csv('movies_bus_semantica.csv', encoding='utf-8-sig')

In [153]:
#sample = movie_df[:15]
#sample

,titulo,link,traduccion,trad_num_car,trad_num_pal
0,Apocalypse Now,https://www.imdb.com/title/tt0078788/?ref_=adv...,'Un oficial del ejército estadounidense que si...,174,31
1,El lago azul,https://www.imdb.com/title/tt0080453/?ref_=adv...,"'En la época victoriana, dos niños naufragan e...",206,34
2,"Alien, el octavo pasajero",https://www.imdb.com/title/tt0078748/?ref_=adv...,'La tripulación de una nave espacial comercial...,134,20
3,El resplandor,https://www.imdb.com/title/tt0081505/?ref_=adv...,'Una familia se dirige a un hotel aislado para...,206,32
4,El Imperio contraataca,https://www.imdb.com/title/tt0080684/?ref_=adv...,'Tras ser brutalmente dominados los rebeldes p...,228,35
5,Aterriza como puedas,https://www.imdb.com/title/tt0080339/?ref_=adv...,'Un hombre con miedo a volar debe asegurarse d...,121,21
6,Los pasajeros del tiempo,https://www.imdb.com/title/tt0080025/?ref_=adv...,'H. G. Wells persigue a Jack el Destripador ha...,166,30
7,Calígula,https://www.imdb.com/title/tt0080491/?ref_=adv...,"'Detalla la historia impactante, pero innegabl...",102,14
8,Holocausto caníbal,https://www.imdb.com/title/tt0078935/?ref_=adv...,'Durante una misión de rescate en la selva ama...,150,23
9,"10, la mujer perfecta",https://www.imdb.com/title/tt0078721/?ref_=adv...,'Un compositor de Hollywood atraviesa una cris...,114,20


In [154]:
#%%time

#normalizamos la descripción
#sample['traduccion_limpia'] = sample['traduccion'].apply(normaliza)
#sample

Wall time: 155 ms


,titulo,link,traduccion,trad_num_car,trad_num_pal,traduccion_limpia
0,Apocalypse Now,https://www.imdb.com/title/tt0078788/?ref_=adv...,'Un oficial del ejército estadounidense que si...,174,31,oficial ejército estadounidense servir vietnam...
1,El lago azul,https://www.imdb.com/title/tt0080453/?ref_=adv...,"'En la época victoriana, dos niños naufragan e...",206,34,época victorián niño naufragar isla tropical p...
2,"Alien, el octavo pasajero",https://www.imdb.com/title/tt0078748/?ref_=adv...,'La tripulación de una nave espacial comercial...,134,20,tripulación nave espacial comercial forma vida...
3,El resplandor,https://www.imdb.com/title/tt0081505/?ref_=adv...,'Una familia se dirige a un hotel aislado para...,206,32,familia dirigir hotel aislado pasar invierno p...
4,El Imperio contraataca,https://www.imdb.com/title/tt0080684/?ref_=adv...,'Tras ser brutalmente dominados los rebeldes p...,228,35,brutalmente dominado rebelde imperio planeta h...
5,Aterriza como puedas,https://www.imdb.com/title/tt0080339/?ref_=adv...,'Un hombre con miedo a volar debe asegurarse d...,121,21,hombre miedo volar asegurar él avión aterrizar...
6,Los pasajeros del tiempo,https://www.imdb.com/title/tt0080025/?ref_=adv...,'H. G. Wells persigue a Jack el Destripador ha...,166,30,persona perseguir persona siglo xx asesino ser...
7,Calígula,https://www.imdb.com/title/tt0080491/?ref_=adv...,"'Detalla la historia impactante, pero innegabl...",102,14,detallar historia impactante innegablemente tr...
8,Holocausto caníbal,https://www.imdb.com/title/tt0078935/?ref_=adv...,'Durante una misión de rescate en la selva ama...,150,23,misión rescate selva amazónico profesor tropie...
9,"10, la mujer perfecta",https://www.imdb.com/title/tt0078721/?ref_=adv...,'Un compositor de Hollywood atraviesa una cris...,114,20,compositor hollywood atravesar crisis mediano ...


Analisis exploratorio de datos después de limpieza (número de carácteres, número de palabras)

In [8]:
#el número de carácteres
movie_df['trad_limpia_num_car'] = movie_df['traduccion_limpia'].str.len()
#el número de palabras
movie_df['trad_limpia_num_pal'] = movie_df['traduccion_limpia'].str.split().str.len()
#count.index = count.index.astype(str)
movie_df.sort_index(inplace=True)

movie_df

,titulo,link,traduccion,trad_num_car,trad_num_pal,traduccion_limpia,trad_limpia_num_car,trad_limpia_num_pal
0,Apocalypse Now,https://www.imdb.com/title/tt0078788/?ref_=adv...,'Un oficial del ejército estadounidense que si...,174,31,uno oficial del ejército estadounidense que se...,169,29
1,El lago azul,https://www.imdb.com/title/tt0080453/?ref_=adv...,"'En la época victoriana, dos niños naufragan e...",206,34,en el época victorián dos niño naufragar en un...,189,34
2,"Alien, el octavo pasajero",https://www.imdb.com/title/tt0078748/?ref_=adv...,'La tripulación de una nave espacial comercial...,134,20,el tripulación de uno nave espacial comercial ...,131,20
3,El resplandor,https://www.imdb.com/title/tt0081505/?ref_=adv...,'Una familia se dirige a un hotel aislado para...,206,32,uno familia él dirigir uno hotel aislado para ...,192,30
4,El Imperio contraataca,https://www.imdb.com/title/tt0080684/?ref_=adv...,'Tras ser brutalmente dominados los rebeldes p...,228,35,tras ser brutalmente dominado el rebelde por e...,213,34
...,...,...,...,...,...,...,...,...
247141,Scream of the Bikini,https://www.imdb.com/title/tt1358254/?ref_=adv...,'Una joven pareja de Nueva York crea una intri...,86,14,uno joven pareja de nueva york creer uno intri...,85,14
247142,Puck Hogs,https://www.imdb.com/title/tt1249307/?ref_=adv...,'Un joven novelista intenta escribir sobre el ...,160,28,uno joven novelista intentar escribir sobre el...,158,28
247143,Caído de las nubes,https://www.imdb.com/title/tt1526741/?ref_=adv...,'Padre e hijo viajando de Victoria a Cape York...,130,24,padre hijo viajar de victoria cape york para c...,120,22
247144,Los Tres Investigadores y el Secreto del Casti...,https://www.imdb.com/title/tt1156519/?ref_=adv...,'Al Lcpl Arne Christenson le quedan tres días ...,148,29,al lcpl arne christenson él quedar tres día pa...,142,27


In [9]:
print("Número promedio de carácteres: " + str(movie_df['trad_limpia_num_car'].mean(axis = 0, skipna = True)))
print("Número promedio de palabras:   " + str(movie_df['trad_limpia_num_pal'].mean(axis = 0, skipna = True)))

Número promedio de carácteres: 214.2350999004637
Número promedio de palabras:   36.75260372411449
